## Dataset Preprocess

In [ ]:
sentence_list =
label_list = 

### Tokenize

In [ ]:
import pickle
import numpy as np

import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize


import re

tokenized_words = []
for sentence in sentence_list:
    clean_sent = re.sub(r'[^\w\s]','', sentence).lower()
    tokenize_word = word_tokenize(clean_sent)
    tokenized_words.append(tokenize_word)

#### seq_length (modify)

In [ ]:
length_list = [len(words) for words in tokenized_words]
print(max(length_list))

seq_length = 
# print(sum([1 for l in length_list if l<512])/len(length_list))
# print(sum([1 for l in length_list if l<256])/len(length_list))
# print(sum([1 for l in length_list if l<128])/len(length_list))

### Word Index Encode

In [ ]:
from keras.preprocessing.sequence import pad_sequences

tokenPAD = "<pad>"
tokenUNK = "<unknown>"

unique_words_temp = set()
for sentence in tokenized_words:
    for word in sentence:
        unique_words_temp.add(word)
unique_words = [tokenPAD, tokenUNK]
unique_words.extend(unique_words_temp)

unique_words_dict = {word: i for i, word in enumerate(unique_words)}


word_index_encode = []
for sentence in tokenized_words:
    word_index_encode_temp = []    
    for word in sentence:
        try:
            word_index_encode_temp.append(unique_words_dict[word])
        except KeyError:
            word_index_encode_temp.append(unique_words_dict[tokenUNK])
    word_index_encode.append(word_index_encode_temp)

#Pad sequences
word_index_encode = pad_sequences(word_index_encode, maxlen = seq_length, value = unique_words_dict[tokenPAD], padding = "post")

### Label Encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder

labels=np.unique(label_list)

num_class = len(labels)
lEnc = LabelEncoder()
lEnc.fit(labels)

encoded_labels = lEnc.transform(label_list)

print(labels)
print(lEnc.transform(labels))

## Embeddings

### Target Specific: NER (modify)

In [ ]:
NER_embedding = np.load('NER_embedding.npy')
print(NER_embedding.shape)

### Target General: GP GN Golve

In [ ]:
gpgn_glove_dict = pickle.load( open("gpgn_glove_dict.pkl", "rb" ) )

In [ ]:
glove_embedding_matrix = np.zeros((len(unique_words), 300))
for i, word in enumerate(unique_words):
    try:
        if word != tokenPAD: #set pad weights to 0
            glove_embedding_matrix[i] = gpgn_glove_dict[word]
    except KeyError:
        pass
        
glove_embedding_matrix.shape

### Content Explicit: Dict2vec

In [ ]:
dict2vec_dict = pickle.load( open( "dict2vec_dict.pkl", "rb" ) )

In [ ]:
dict2vec_embedding_matrix = np.zeros((len(unique_words), 300))
for i, word in enumerate(unique_words):
    try:
        if word != tokenPAD: #set pad weights to 0
            dict2vec_embedding_matrix[i] = dict2vec_dict[word]
    except KeyError:
        pass
        
dict2vec_embedding_matrix.shape

### Content Implicit: Sarcasm (modify)

In [ ]:
sarcasm_embedding_list = []

with open("sarcasm_embedding.txt","r") as f:
  raw_string = f.readline()
  raw_string = raw_string[1:-1]
  
  raw_lists = raw_string.split('[')
  for l in raw_lists[1:]:
    l = l[:-3]
    float_list = l.split(",")
    float_list = [float(f) for f in float_list]
    sarcasm_embedding_list.append(float_list+[0]*7)
sarcasm_embedding=np.array(sarcasm_embedding_list)   
del sarcasm_embedding_list 

sarcasm_embedding=sarcasm_embedding.reshape((sarcasm_embedding.shape[0],1,-1))
sarcasm_embedding=np.repeat(sarcasm_embedding,seq_length,1)
print(sarcasm_embedding.shape)

### GCN

In [ ]:
import json

dict2vec_dict = json.load( open( "gcn_dict.json", "rb" ) )

gcn_embedding_matrix = np.zeros((len(unique_words), 200))
for i, word in enumerate(unique_words):
    try:
        if word != tokenPAD: #set pad weights to 0
            gcn_embedding_matrix[i] = dict2vec_dict[word]
    except KeyError:
        pass
        
gcn_embedding_matrix.shape

## Train Test Split

In [ ]:
from sklearn.model_selection import train_test_split
index_train, index_test = train_test_split(word_index_encode, test_size=0.1, random_state=28)
NER_train, NER_test = train_test_split(NER_embedding, test_size=0.1, random_state=28)
sarcasm_train, sarcasm_test = train_test_split(sarcasm_embedding, test_size=0.1, random_state=28)
label_train, label_test = train_test_split(encoded_labels, test_size=0.1, random_state=28)

## Model

In [ ]:
!pip install keras-layer-normalization
from keras_layer_normalization import LayerNormalization
from datetime import datetime
import keras.utils
import keras.activations
import keras.applications
import keras.backend
import keras.datasets
import keras.engine
import keras.layers
import keras.preprocessing
import keras.wrappers
import keras.callbacks
import keras.constraints
import keras.initializers
import keras.metrics
import keras.models
import keras.losses
import keras.optimizers
import keras.regularizers 
from keras.optimizers import Adam

In [ ]:
from keras import backend as K
import math 


def matmul(x):
    return tf.matmul(x[0], x[1])

def split_heads(x,num_heads):
    x = keras.layers.Reshape((seq_length, num_heads, -1))(x)
    return keras.layers.Permute((2,1,3))(x)

def scaled_dot_product_attention(q, k, v, depth):
  matmul_qk = keras.layers.Lambda(matmul)([q,keras.layers.Permute((1,3,2))(k)]) 
  # scale matmul_qk
  scaled_attention_logits= keras.layers.Lambda(lambda x:x/math.sqrt(depth))(matmul_qk)
  # softmax is normalized on the last axis (seq_len_k) so that the scores
  # add up to 1.
  attention_weights = keras.layers.Activation('softmax')(scaled_attention_logits)  # (..., seq_len_q, seq_len_k)
  output = keras.layers.Lambda(matmul)([attention_weights, v])
  return output

def new_trans(input_target_content,d_model,num_heads,dff,dropout):
  input_target = input_target_content[0]
  input_content = input_target_content[1]
  depth = d_model/num_heads

  q_target_1 = keras.layers.Dense(d_model)(input_target)
  k_target_1 = keras.layers.Dense(d_model)(input_target)
  v_target_1 = keras.layers.Dense(d_model)(input_target)
  q_content_1 = keras.layers.Dense(d_model)(input_content)
  k_content_1 = keras.layers.Dense(d_model)(input_content)
  v_content_1 = keras.layers.Dense(d_model)(input_content)

  qw_target_1 = split_heads(q_target_1,num_heads)  
  kw_target_1 = split_heads(k_target_1,num_heads)  
  vw_target_1 = split_heads(v_target_1,num_heads) 

  qw_content_1 = split_heads(q_content_1,num_heads)  
  kw_content_1 = split_heads(k_content_1,num_heads)  
  vw_content_1 = split_heads(v_content_1,num_heads) 

  scaled_attention_content = scaled_dot_product_attention(qw_content_1, kw_target_1, vw_target_1, depth)
  scaled_attention_content = keras.layers.Permute((2,1,3))(scaled_attention_content)  
  concat_attention_content = keras.layers.Reshape((-1, d_model))(scaled_attention_content)
  concat_attention_content = keras.layers.Dense(d_model)(concat_attention_content)
  concat_attention_content = keras.layers.Dropout(dropout)(concat_attention_content)
  out_1_content = LayerNormalization()(keras.layers.Add()([q_content_1,concat_attention_content]))
  ffn_output_content = keras.layers.Dense(d_model)(keras.layers.Dense(dff, activation='relu')(out_1_content))
  ffn_output_content = keras.layers.Dropout(dropout)(ffn_output_content)
  out_2_content = LayerNormalization()(keras.layers.Add()([ffn_output_content,out_1_content]))


  scaled_attention_target = scaled_dot_product_attention(qw_target_1, kw_content_1, vw_content_1, depth)
  scaled_attention_target = keras.layers.Permute((2,1,3))(scaled_attention_target)
  concat_attention_target = keras.layers.Reshape((-1, d_model))(scaled_attention_target)
  concat_attention_target = keras.layers.Dense(d_model)(concat_attention_target)
  concat_attention_target = keras.layers.Dropout(dropout)(concat_attention_target)
  out_1_target = LayerNormalization()(keras.layers.Add()([q_target_1,concat_attention_target]))
  ffn_output_target = keras.layers.Dense(d_model)(keras.layers.Dense(dff, activation='relu')(out_1_target))
  ffn_output_target = keras.layers.Dropout(dropout)(ffn_output_target)
  out_2_target = LayerNormalization()(keras.layers.Add()([ffn_output_target,out_1_target]))

  return out_2_target,out_2_content

def trans(input_emb, d_model,num_heads,dff,dropout):

  depth = d_model/num_heads

  q = keras.layers.Dense(d_model)(input_emb)
  k = keras.layers.Dense(d_model)(input_emb)
  v = keras.layers.Dense(d_model)(input_emb)

  qw = split_heads(q,num_heads)  
  kw = split_heads(k,num_heads)  
  vw = split_heads(v,num_heads) 

  scaled_attention = scaled_dot_product_attention(qw, kw, vw, depth)
  scaled_attention = keras.layers.Permute((2,1,3))(scaled_attention)  
  concat_attention = keras.layers.Reshape((-1, d_model))(scaled_attention)
  concat_attention = keras.layers.Dense(d_model)(concat_attention)
  concat_attention = keras.layers.Dropout(dropout)(concat_attention)
  out_1 = LayerNormalization()(keras.layers.Add()([q,concat_attention]))
  ffn_output = keras.layers.Dense(d_model)(keras.layers.Dense(dff, activation='relu')(out_1))
  ffn_output = keras.layers.Dropout(dropout)(ffn_output)
  out_2 = LayerNormalization()(keras.layers.Add()([ffn_output,out_1]))

  return out_2

In [ ]:
def get_model(use_gcn = 3):   

    input_index = keras.layers.Input(shape = (index_train.shape[1],))
    glove_emb =  keras.layers.Embedding(input_dim = glove_embedding_matrix.shape[0], output_dim = glove_embedding_matrix.shape[1], input_length = seq_length, weights = [glove_embedding_matrix], trainable = False)(input_index)
    dict2vec_emb = keras.layers.Embedding(input_dim = dict2vec_embedding_matrix.shape[0], output_dim = dict2vec_embedding_matrix.shape[1], input_length = seq_length, weights = [dict2vec_embedding_matrix], trainable = False)(input_index)
    gcn_emb = keras.layers.Embedding(input_dim = gcn_embedding_matrix.shape[0], output_dim = gcn_embedding_matrix.shape[1], input_length = seq_length, weights = [gcn_embedding_matrix], trainable = False)(input_index)
    
    input_NER = keras.layers.Input(shape=(seq_length,9))
    input_sar = keras.layers.Input(shape=(seq_length,9))

    input_target=keras.layers.Concatenate(axis=-1)([glove_emb,input_NER])
    input_content=keras.layers.Concatenate(axis=-1)([dict2vec_emb,input_sar])

    target_out, content_out = new_trans([input_target,input_content],d_model=309, num_heads=3, dff = 1296, dropout=0.5)
    target_out_2, content_out_2 = new_trans([target_out,content_out],d_model=309, num_heads=3, dff = 1296, dropout=0.5)
    encoded_output_tar = keras.layers.Dropout(0.1)(target_out_2)
    encoded_output_lan = keras.layers.Dropout(0.1)(content_out_2)
 
    gcn_encoder_out = trans(gcn_emb, d_model=200, num_heads=2, dff = 1296, dropout=0.5)
    gcn_encoder_out_2 = trans(gcn_encoder_out, d_model=200, num_heads=2, dff = 1296, dropout=0.5)
    encoded_output_gcn = keras.layers.Dropout(0.1)(gcn_encoder_out_2)

    encoded_output_gcn_2 = keras.layers.AveragePooling1D(pool_size=3, strides=1)(encoded_output_gcn)
    encoded_output_gcn_3 = keras.layers.AveragePooling1D(pool_size=3, strides=1)(encoded_output_gcn_2)
    encoded_output_gcn_4 = keras.layers.AveragePooling1D(pool_size=3, strides=1)(encoded_output_gcn_3)

    ######  lan-to-tar (target-gate) ######
    hidden_size_lan = encoded_output_lan.shape[-1].value
    sequence_length = encoded_output_lan.shape[-2].value
    
    #first
    #target gate input preperation
    flatten_tar_gate_input = keras.layers.Flatten()(encoded_output_lan)
    log_probs_tar_gate_input = keras.layers.Dense(3, input_shape=(sequence_length, hidden_size_lan), activation='softmax')(flatten_tar_gate_input)
    
    lan_repeat_tar_gate = keras.layers.RepeatVector(sequence_length)(log_probs_tar_gate_input)
    tar_gate_input_concat = keras.layers.Concatenate(axis=-1)([encoded_output_tar, lan_repeat_tar_gate])
        
    #gating 
    hidden_size_tar_gate_input_concat = tar_gate_input_concat.shape[-1].value

    tar_concat_1 = keras.layers.Concatenate(axis=-1)([tar_gate_input_concat, encoded_output_gcn])
    tar_cnn_1 =  keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu')(tar_concat_1)

    
    ######  tar-to-lan (language-gate) ######
    hidden_size_tar = encoded_output_tar.shape[-1].value
    
    #language gate input preperation 
    flatten_lan_gate_input = keras.layers.Flatten()(encoded_output_tar)
    log_probs_lan_gate_input = keras.layers.Dense(3, input_shape=(sequence_length, hidden_size_tar), activation='softmax')(flatten_lan_gate_input)
    
    
    tar_repeat_lan_gate = keras.layers.RepeatVector(sequence_length)(log_probs_lan_gate_input)
    lan_gate_input_concat = keras.layers.Concatenate(axis=-1)([encoded_output_lan, tar_repeat_lan_gate])
    
    #gating
    hidden_size_lan_gate_input_concat= lan_gate_input_concat.shape[-1].value

    lan_concat_1 = keras.layers.Concatenate(axis=-1)([lan_gate_input_concat, encoded_output_gcn])
    lan_cnn_1 =  keras.layers.Conv1D(filters=128, kernel_size=3, activation='relu')(lan_concat_1)
    
    lan_cnn_1_size = lan_cnn_1.shape[-2].value
    flatten_tar_cnn_1 = keras.layers.Flatten()(tar_cnn_1)
    log_probs_tar_cnn_1 = keras.layers.Dense(3, activation='softmax')(flatten_tar_cnn_1)
    tar_repeat_lan_gate_1 = keras.layers.RepeatVector(lan_cnn_1_size)(log_probs_tar_cnn_1)
    tar_gate_input_concat_1 = keras.layers.Concatenate(axis=-1)([lan_cnn_1, tar_repeat_lan_gate_1])

    flatten_lan_cnn_1 = keras.layers.Flatten()(lan_cnn_1)
    log_probs_lan_cnn_1 = keras.layers.Dense(3, activation='softmax')(flatten_lan_cnn_1)
    lan_repeat_tar_gate_1 = keras.layers.RepeatVector(lan_cnn_1_size)(log_probs_lan_cnn_1)
    lan_gate_input_concat_1 = keras.layers.Concatenate(axis=-1)([tar_cnn_1, lan_repeat_tar_gate_1])

    #second
    lan_concat_1_size = lan_gate_input_concat_1.shape[-1].value
    encoded_output_gcn_2 = keras.layers.Dense(lan_concat_1_size)(encoded_output_gcn_2)

    tar_concat_2 = keras.layers.Concatenate(axis=-1)([tar_gate_input_concat_1, encoded_output_gcn_2])
    tar_cnn_2 =  keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(tar_concat_2)

    lan_concat_2 = keras.layers.Concatenate(axis=-1)([lan_gate_input_concat_1, encoded_output_gcn_2])
    lan_cnn_2 =  keras.layers.Conv1D(filters=64, kernel_size=3, activation='relu')(lan_concat_2)

    lan_cnn_2_size = lan_cnn_2.shape[-2].value
    flatten_tar_cnn_2 = keras.layers.Flatten()(tar_cnn_2)
    log_probs_tar_cnn_2 = keras.layers.Dense(3, activation='softmax')(flatten_tar_cnn_2)
    tar_repeat_lan_gate_2 = keras.layers.RepeatVector(lan_cnn_2_size)(log_probs_tar_cnn_2)
    tar_gate_input_concat_2 = keras.layers.Concatenate(axis=-1)([lan_cnn_2, tar_repeat_lan_gate_2])

    flatten_lan_cnn_2 = keras.layers.Flatten()(lan_cnn_2)
    log_probs_lan_cnn_2 = keras.layers.Dense(3, activation='softmax')(flatten_lan_cnn_2)
    lan_repeat_tar_gate_2 = keras.layers.RepeatVector(lan_cnn_2_size)(log_probs_lan_cnn_2)
    lan_gate_input_concat_2 = keras.layers.Concatenate(axis=-1)([tar_cnn_2, lan_repeat_tar_gate_2])

    #third
    lan_concat_2_size = lan_gate_input_concat_2.shape[-1].value
    encoded_output_gcn_3 = keras.layers.Dense(lan_concat_2_size)(encoded_output_gcn_3)

    tar_concat_3 = keras.layers.Concatenate(axis=-1)([tar_gate_input_concat_2, encoded_output_gcn_3])
    tar_cnn_3 =  keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(tar_concat_3)

    lan_concat_3 = keras.layers.Concatenate(axis=-1)([lan_gate_input_concat_2, encoded_output_gcn_3])
    lan_cnn_3 =  keras.layers.Conv1D(filters=32, kernel_size=3, activation='relu')(lan_concat_3)

    lan_cnn_3_size = lan_cnn_3.shape[-2].value
    flatten_tar_cnn_3 = keras.layers.Flatten()(tar_cnn_3)
    log_probs_tar_cnn_3 = keras.layers.Dense(3, activation='softmax')(flatten_tar_cnn_3)
    tar_repeat_lan_gate_3 = keras.layers.RepeatVector(lan_cnn_3_size)(log_probs_tar_cnn_3)
    tar_gate_input_concat_3 = keras.layers.Concatenate(axis=-1)([lan_cnn_3, tar_repeat_lan_gate_3])

    flatten_lan_cnn_3 = keras.layers.Flatten()(lan_cnn_3)
    log_probs_lan_cnn_3 = keras.layers.Dense(3, activation='softmax')(flatten_lan_cnn_3)
    lan_repeat_tar_gate_3 = keras.layers.RepeatVector(lan_cnn_3_size)(log_probs_lan_cnn_3)
    lan_gate_input_concat_3 = keras.layers.Concatenate(axis=-1)([tar_cnn_3, lan_repeat_tar_gate_3])


    #concat
    if use_gcn == 0:
        concat_result = keras.layers.Concatenate(axis=-1)([tar_gate_input_concat, lan_gate_input_concat])
    elif use_gcn == 1:       
        concat_result = keras.layers.Concatenate(axis=-1)([tar_gate_input_concat_1, lan_gate_input_concat_1])
    elif use_gcn == 2:
        concat_result = keras.layers.Concatenate(axis=-1)([tar_gate_input_concat_2, lan_gate_input_concat_2])
    elif use_gcn == 3:
        concat_result = keras.layers.Concatenate(axis=-1)([tar_gate_input_concat_3, lan_gate_input_concat_3])
        
    #classification
    flatten = keras.layers.Flatten()(concat_result)
    mlp_1 = keras.layers.Dense(512, activation="relu")(flatten)
    mlp_2 = keras.layers.Dense(64, activation="relu")(mlp_1)
    # mlp_3 = keras.layers.Dense(16, activation="relu")(mlp_2)
    dense_layer = keras.layers.Dense(2, activation="softmax")(mlp_2)
    
    return keras.models.Model(inputs=[input_index, input_NER, input_sar], outputs=dense_layer)

In [ ]:
from datetime import datetime

def evaluate(trans_model, index_train, index_test, NER_train, NER_test, sarcasm_train, sarcasm_test, label_train, label_test, epochs, learningRate):
  
  trans_model.compile(optimizer= Adam(learningRate),loss='sparse_categorical_crossentropy',metrics = ["accuracy"])  
  trans_model.summary()

  current_time = datetime.now()
  trans_model.fit([index_train, NER_train, sarcasm_train], label_train, batch_size=batch_size, epochs=epochs)
  train_time=datetime.now() - current_time
  print("Training took time: ", train_time)


  #Predict classes
  preds = trans_model.predict([index_test, NER_test, sarcasm_test], batch_size = batch_size)

  return np.argmax(preds, axis=-1)

In [ ]:
from google.colab import files
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sn
from tabulate import tabulate
import warnings
warnings.filterwarnings("ignore")

def plotConfusionMatrix(cmArray, labels, title, download=False):
    df_cm = pd.DataFrame(cmArray, index = [i for i in labels],columns = [i for i in labels])
  
    plt.figure(figsize = (5,4))
    #plt.title(title)
    sn.heatmap(df_cm, annot=True, cmap="YlGnBu", fmt='g')
    if (download):
        plt.savefig(title)
        files.download(title+".png")

In [ ]:

epochs=7
batch_size = 16
learningRate=1e-6

trans_model=get_model(use_gcn = 3)

preds = evaluate(trans_model, index_train, index_test, NER_train, NER_test, sarcasm_train, sarcasm_test, label_train, label_test, epochs, learningRate)

In [ ]:
print(classification_report(lEnc.inverse_transform(label_test), lEnc.inverse_transform(preds),digits=4))

cmArray = confusion_matrix(label_test, preds)
print(pd.DataFrame(cmArray, columns = labels, index = labels))
plotConfusionMatrix(cmArray, labels, "20ng, 4e-5, epoch "+str(epochs), False)  